Import libraries

In [6]:
import cv2
import dlib
import numpy as np

Define class

In [7]:
class ClosedEyesDetector:
    
    def __init__(self):
        
        self.landmarks = None
        self.left_eye = None
        self.right_eye = None
        
        # eye aspect ratio
        self.left_EAR = None
        self.right_EAR = None
        
        
    def update_eye(self, landmarks):
        
        self.landmarks = landmarks
        
        # The landmarks for the left eye are 36-41
        self.left_eye = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)], np.int32)
        # The landmarks for the right eye are 42-47
        self.right_eye = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)], np.int32)
        
        self.left_EAR = self._calculate_eye_aspect_ratio(self.left_eye)
        self.right_EAR = self._calculate_eye_aspect_ratio(self.right_eye)
    
    def is_closed(self):
        return True if self.left_EAR < 0.2 or self.right_EAR < 0.2 else False
    
    def _calculate_eye_aspect_ratio(self, eye):
        # compute the euclidean distances between the two sets of
        # vertical eye landmarks (x, y)-coordinates
        A = np.linalg.norm(eye[1] - eye[5])
        B = np.linalg.norm(eye[2] - eye[4])

        # compute the euclidean distance between the horizontal
        # eye landmark (x, y)-coordinates
        C = np.linalg.norm(eye[0] - eye[3])

        # compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)

        return ear

Run Script

In [8]:
# use dlib pre-trained face detection
detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

cap = cv2.VideoCapture(0)

closed_eye_detector = ClosedEyesDetector()

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:

        landmarks = predictor(gray, face)
        
        closed_eye_detector.update_eye(landmarks)

        cv2.polylines(frame, [closed_eye_detector.left_eye], True, (0, 255, 0), 1)
        cv2.polylines(frame, [closed_eye_detector.right_eye], True, (0, 255, 0), 1)
        
        if closed_eye_detector.is_closed():
            cv2.putText(frame, "Eyes closed", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    # escape to quit
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()